In [101]:
import csv
import numpy as np
from collections import defaultdict

# Data extraction
### Source
Raw csv can be found locally [here](./tus_00week_1_Data.csv) and remotly on Eurostat website [here](http://appsso.eurostat.ec.europa.eu/nui/submitViewTableAction.do) .  
See also [explanatory metadata](https://ec.europa.eu/eurostat/cache/metadata/en/tus_esms.htm).
### Extract data from csv into nested dict

In [102]:
with open('tus_00week_1_Data.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    print("Initial fields are :", ', '.join(next(csv_reader)), '\n' ) 
    
    Nest_dict = lambda: defaultdict(Nest_dict)
    nest_dict = Nest_dict()
    for row in csv_reader:
        time_string = row[6]
        # select Time spent only (not Participation time nor Participation rate)
        if row[0]=="Time spent (hh:mm)":
            try:
                hours, minutes = map(int, time_string.split(':'))
                nest_dict[row[1]][row[2]][row[3]][row[4]][row[5]] = hours * 60 + minutes     
            except Exception as err: 
                nest_dict[row[1]][row[2]][row[3]][row[4]][row[5]] = None         
    print("Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00") 

Initial fields are : UNIT, GEO, TIME, DAYSWEEK, SEX, ACL00, Value, Flag and Footnotes 

Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00


### Convert nested dict into ndarray and save key names

In [103]:
def convert_nest_dict_2_list(nest_dict):
    local_list = []
    if not isinstance(nest_dict, defaultdict):
        return nest_dict
    for value in nest_dict.values():
        local_list.append(convert_nest_dict_2_list(value))
    return local_list

In [104]:
data_2y_array = np.asarray(convert_nest_dict_2_list(nest_dict), dtype=object)
print("Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00") 
print(f"Data shape: {data_2y_array.shape}.\
 There is NoneTyped data left: {np.any(data_2y_array==None)}")

Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00
Data shape: (21, 2, 3, 3, 56). There is NoneTyped data left: True


In [105]:
keys_list = []
def convert_nest_dict_2_list(nest_dict):
    if not isinstance(nest_dict, defaultdict):
        return 1
    keys_list.append(list(nest_dict.keys()))
    first_value = list(nest_dict.values())[0]
    convert_nest_dict_2_list(first_value)
        
convert_nest_dict_2_list(nest_dict)
print("Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00")
print(keys_list)

Current fields are : GEO, TIME, DAYSWEEK, SEX, ACL00
[['Belgium', 'Bulgaria', 'Germany (until 1990 former territory of the FRG)', 'Estonia', 'Greece', 'Spain', 'France', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Netherlands', 'Austria', 'Poland', 'Romania', 'Slovenia', 'Finland', 'Norway', 'United Kingdom', 'Serbia'], ['2000', '2010'], ['All days of the week', 'Monday to Friday', 'Saturday to Sunday'], ['Total', 'Males', 'Females'], ['Total', 'Personal care', 'Sleep', 'Eating', 'Other and/or unspecified personal care', 'Employment, related activities and travel as part of/during main and second job', 'Main and second job and related travel', 'Activities related to employment and unspecified employment', 'Study', 'School and university except homework', 'Homework', 'Free time study', 'Household and family care', 'Food management except dish washing', 'Dish washing', 'Cleaning dwelling', 'Household upkeep except cleaning dwelling', 'Laundry', 'Ironing', 'Handicraft and pr

### Aggregate TIME (2000 and 2010) by selecting most recent available data

In [106]:
data_array = np.array([ data_2y_array[k,0,:,:,:]
                       if np.any(data_2y_array[k,1,:,:,:]==None)
                       else data_2y_array[k,1,:,:,:]
                       for k in range(data_2y_array.shape[0])])
print("Current fields are : GEO, DAYSWEEK, SEX, ACL00") 
print(f"Data shape: {data_array.shape}. There is NoneTyped data left: {np.any(data_array==None)}")

Current fields are : GEO, DAYSWEEK, SEX, ACL00
Data shape: (21, 3, 3, 56). There is NoneTyped data left: False


### Fields description of data_array

* GEO
| Index  |                  Code             |                  Label                                                       |
|----|----------------------------------|--------------------------------------------------------------------------------|
| 0  |                  BE              |                  Belgium                                                       |
| 1  |                  BG              |                  Bulgaria                                                      |
| 2  |                  DE              |                  Germany (until 1990 former territory of the FRG)              |
| 3  |                  EE              |                  Estonia                                                       |
| 4  |                  EL              |                  Greece                                                        |
| 5  |                  ES              |                  Spain                                                         |
| 6  |                  FR              |                  France                                                        |
| 7  |                  IT              |                  Italy                                                         |
| 8  |                  LV              |                  Latvia                                                        |
| 9  |                  LT              |                  Lithuania                                                     |
| 10 |                  LU              |                  Luxembourg                                                    |
| 11 |                  HU              |                  Hungary                                                       |
| 12 |                  NL              |                  Netherlands                                                   |
| 13 |                  AT              |                  Austria                                                       |
| 14 |                  PL              |                  Poland                                                        |
| 15 |                  RO              |                  Romania                                                       |
| 16 |                  SI              |                  Slovenia                                                      |
| 17 |                  FI              |                  Finland                                                       |
| 18 |                  NO              |                  Norway                                                        |
| 19 |                  UK              |                  United Kingdom                                                |
| 20 |                  RS              |                  Serbia                                                        |

<br>

* DAYSWEEK
|Index|Code|Label|
|---|------|----------------------|
| 0 | D1-7 | All days of the week |
| 1 | D1-5 | Monday to Friday     |
| 2 | D6-7 | Saturday to Sunday   |

<br>

* SEX
|Index|              Code             |                  Label                |
|---|---------------------------------|---------------------------------------|
| 0 |                  T              |                  Total                |
| 1 |                  M              |                  Males                |
| 2 |                  F              |                  Females              |

<br>

* ACL00 (activities)
| Index|      Code           |       Label                                                                      |                     Sub-Label                                    |
|----|-----------------------|-----------------------------------------------------------------------------------|-----------------------------------------------------------------|
| 0  |        TOTAL          | Total                                                                             |                                                                 |
| 1  |        AC0            | Personal   care                                                                   |                                                                 |
| 2  |        AC01           |    ├                                                                             | Sleep                                                           |
| 3  |        AC02           | ├                                                                                 | Eating                                                          |
| 4  |        AC03           | └                                                                                 | Other and/or unspecified   personal care                        |
| 5  |        AC1_TR         | Employment,   related activities and travel<br> as part of/during main and second job |                                                                 |
| 6  |        AC1A           | ├                                                                                 | Main and second job and   related travel                        |
| 7  |        AC1B           | └                                                                                 | Activities   related to employment and unspecified employment   |
| 8  |        AC2            | Study                                                                             |                                                                 |
| 9  |        AC21A          | ├                                                                                 | School and university except   homework                         |
| 10 |        AC212          | ├                                                                                 | Homework                                                        |
| 11 |        AC22           | └                                                                                 | Free time   study                                               |
| 12 |        AC3            | Household   and family care                                                       |                                                                 |
| 13 |        AC31A          | ├                                                                                 | Food management except dish   washing                           |
| 14 |        AC313          | ├                                                                                 | Dish washing                                                    |
| 15 |        AC321          | ├                                                                                 | Cleaning   dwelling                                             |
| 16 |        AC32A          | ├                                                                                 | Household upkeep except   cleaning dwelling                     |
| 17 |        AC331          | ├                                                                                 | Laundry                                                         |
| 18 |        AC332          | ├                                                                                 | Ironing                                                         |
| 19 |        AC33A          | ├                                                                                 | Handicraft   and producing textiles and other care for textiles |
| 20 |        AC34A          | ├                                                                                 | Gardening;   other pet care                                     |
| 21 |        AC342          | ├                                                                                 | Tending   domestic animals                                      |
| 22 |        AC343          | ├                                                                                 | Caring for   pets                                               |
| 23 |        AC344          | ├                                                                                 | Walking   the dog                                               |
| 24 |        AC35           | ├                                                                                 | Construction   and repairs                                      |
| 25 |        AC36           | ├                                                                                 | Shopping   and services                                         |
| 26 |        AC38A          | ├                                                                                 | Childcare, except   teaching, reading and talking               |
| 27 |        AC38B          | ├                                                                                 | Teaching, reading and   talking with child                      |
| 28 |        AC37_39        | └                                                                                 | Household management and   help family member                   |
| 29 |        AC4-8          | Leisure, social and associative   life                                            |                                                                 |
| 30 |        AC41           | ├                                                                                 | Organisational   work                                           |
| 31 |        AC42           | ├                                                                                 | Informal help to other households                               |
| 32 |        AC43           | ├                                                                                 | Participatory   activities                                      |
| 33 |        AC51A          | ├                                                                                 | Visiting   and feasts                                           |
| 34 |        AC51B          | ├                                                                                 | Other   social life                                             |
| 35 |        AC52           | ├                                                                                 | Entertainment   and culture                                     |
| 36 |        AC53           | ├                                                                                 | Resting                                                         |
| 37 |        AC611          | ├                                                                                 | Walking   and hiking                                            |
| 38 |        AC6A           | ├                                                                                 | Sports and   outdoor activities except walking and hiking       |
| 39 |        AC733          | ├                                                                                 | Computer   games                                                |
| 40 |        AC7A           | ├                                                                                 | Computing                                                       |
| 41 |        AC7B           | ├                                                                                 | Hobbies and   games except computing and computer games         |
| 42 |        AC812          | ├                                                                                 | Reading   books                                                 |
| 43 |        AC811          | ├                                                                                 | Reading,   except books                                         |
| 44 |        AC82           | ├                                                                                 | TV and video                                                    |
| 45 |        AC83           | ├                                                                                 | Radio and   music                                               |
| 46 |        AC4-8NSP       | └                                                                                 | Unspecified   leisure                                           |
| 47 |        AC9A           | Travel except travel related to   jobs                                            |                                                                 |
| 48 |        AC913          | ├                                                                                 | Travel   to/from work                                           |
| 49 |        AC9B           | ├                                                                                 | Travel   related to study                                       |
| 50 |        AC936          | ├                                                                                 | Travel related to shopping   and services                       |
| 51 |        AC938          | ├                                                                                 | Transporting   a child                                          |
| 52 |        AC9C           | ├                                                                                 | Travel related to other   household purposes                    |
| 53 |        AC9D           | ├                                                                                 | Travel related   to leisure, social and associative life        |
| 54 |        AC90NSP        | ├                                                                                 | Unspecified   travel                                            |
| 55 |        AC99NSP        | └                                                                                 | Unspecified   time use                                          |

### Export as csv

In [107]:
# example : export Belgium, All days of the week, 2D array
np.savetxt("foo.csv", data_array[0,0,:,:], delimiter=",")

In [109]:
GEO = keys_list[0]
DAYSWEEK, SEX, ACL00 = keys_list[2:]

with open('data.csv', 'w', newline="") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    shape = data_array.shape
    csv_writer.writerow(["GEO","DAYSWEEK","SEX","ACL00","Value"])
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                for l in range(shape[3]):
                    csv_writer.writerow([GEO[i], DAYSWEEK[j], SEX[k], ACL00[l], 
                                         str(data_array[i,j,k,l]) ])